## Install Required Packages

In [ ]:
!pip install wget #python package to download files from internet
#langchain a software development framework designed to simplify the creation of applications using large language models
!pip install langchain
!pip install faiss-cpu
#tiktoken is a fast BPE tokeniser for use with OpenAI's models
!pip install tiktoken
#PyPDF2 is a free and open-source pure-python PDF library capable of splitting, merging, cropping, and transforming the pages of PDF files
!pip install PyPDF2
#we will use wget to download our pdf file
!pip install wget
# to work with openAI
!pip install openai

  Preparing metadata (setup.py) ... done
  Created wheel for wget: filename=wget-3.2-py3-none-any.whl size=9655 sha256=67e87c58e4080a6a46083050b6e06fbcd8754f563916f380af3bcc36e22b21d0
  Stored in directory: /root/.cache/pip/wheels/8b/f1/7f/5c94f0a7a505ca1c81cd1d9208ae2064675d97582078e6c769
Successfully built wget
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 18.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 5.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.6/17.6 MB 51.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 20.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 4.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.0/77.0 kB 2.0 MB/s eta 0:00:00


## Import packages

In [ ]:
import wget
from langchain.embeddings.openai import OpenAIEmbeddings
from PyPDF2 import PdfReader
from langchain.text_splitter import CharacterTextSplitter
from langchain.vectorstores import ElasticVectorSearch, Pinecone, Weaviate, FAISS
import wget

## set your OpenAI key

In [ ]:
#set your Open-AI key
#key="you_openAI_kEY"
import os
OPENAI_API_KEY=

## Download your pdf file

In [ ]:
url="https://arxiv.org/pdf/2010.11929"
url="https://www.getinge.com/dam/hospital/documents/english/getingeonlineact_brochure_en_nonus_mx-7878-en-non_us.pdf"
fileName=wget.download(url)


## Read, split and econde text from pdf

In [ ]:
# read data from the file and put them into a variable called pdf_text
reader = PdfReader(fileName)
pdf_text = ''
for page in (reader.pages):
    text = page.extract_text()
    if text:
        pdf_text += text

In [ ]:
pdf_text

'Getinge Online\nAnytime. Anywhere.\nThis document is intended to provide information to an international audience outside of the US.\nConnecting \npeople Connecting \nideasConnecting \nservicesConnecting \nproducts\nGETINGE ONLINE ACUTE CARE PRODUCTS2Improve uptime  \nin the ICU and OR\nGetinge Online connects people, products, services and ideas.  \nIt gives digital access to equipment data, enabling you, your team \nand hospital to gain insights  and maximize uptime . All to improve \nefficiency  in the intensive care unit and operating room. Be \nproactive, save valuable time and focus on the tasks that matters \nmost – connect your Servo-u/n/air ventilators and Flow Family \nanesthesia machines to Getinge Online.\nEquipment data when you need it\nBy connecting your Getinge devices, you allow your \ndepartment to make use of intelligence and knowledge \nwithin equipment. A digital service developed in close \ncollaboration with our customers, biomeds and clinicians, \nto help you t

In [ ]:
# Define our text splitter
text_splitter = CharacterTextSplitter(
    separator = "\n",
    chunk_size = 1000, #thousand charctere
    chunk_overlap  = 200,
    length_function = len,
)


In [ ]:
#Apply splitting
text_chunks = text_splitter.split_text(pdf_text)

In [ ]:
len(text_chunks)

6

In [ ]:
text_chunks[4]

'portal to view and analyze data from a  \nrole-based dashboard.Getinge is a leading global provider of innovative solutions for operating rooms, intensive care units, hospital wards,  \nsterilization departments and for life science companies and institutions. Based on our firsthand experience and close \npartnerships, we are improving the everyday life for people - today and tomorrow. \nThe brochure is intended to provide information to an international audience outside of the US. \nDepending on national regulations, the product offering and indications can differ from market to market, please check \nwith your local Getinge sales team. The presented values are for demonstration only and do not reflect a real clinical case.\nManufacturer · Maquet Critical Care AB · Röntgenvägen 2 SE-171 54 Solna · Sweden · +46 (0)10 335 73 00\nwww.getinge.com\n© Maquet Critical Care AB 2020 . Servo-air, Servo-n and Servo-u are trademarks by Maquet Critical Care AB . MX-7878 rev01QR code to Getinge On

In [ ]:
# Use embeddings from OpenAI
embeddings = OpenAIEmbeddings(openai_api_key="")

In [ ]:
#Convert text to embeddings
pdf_embeddings = FAISS.from_texts(text_chunks, embeddings)
print(type(pdf_embeddings))

<class 'langchain.vectorstores.faiss.FAISS'>


In [ ]:
from langchain.chains.question_answering import load_qa_chain
from langchain.llms import OpenAI

In [ ]:
chain = load_qa_chain(OpenAI(openai_api_key=""), chain_type="stuff")

## Start chatting with the pdf

In [ ]:
query = "what is the title of the article?"
docs = pdf_embeddings.similarity_search(query)
chain.run(input_documents=docs, question=query)

' Getinge Online: Connecting People, Products, Services and Ideas'

In [ ]:
query = "who are the authors of the article?"
docs = pdf_embeddings.similarity_search(query)
chain.run(input_documents=docs, question=query)

' Maquet Critical Care AB.'

In [ ]:
query = "What is the summary of the document?"
docs = pdf_embeddings.similarity_search(query)
chain.run(input_documents=docs, question=query)

' Getinge Online is a digital service that connects people, products, services, and ideas in order to improve efficiency and maximize uptime in intensive care units and operating rooms. It provides access to equipment data, allowing customers, their teams, and hospitals to gain insights and be more proactive.'

In [ ]:
query = "How was the model accuracy?"
docs = pdf_embeddings.similarity_search(query)
chain.run(input_documents=docs, question=query)

' The model accuracy was reported to be between 70-90%, depending on the dataset.'

In [ ]:
query = "what was the the training dataset?"
docs = pdf_embeddings.similarity_search(query)
chain.run(input_documents=docs, question=query)

In [ ]:
query = "How is this model different from other models?"
docs = pdf_embeddings.similarity_search(query)
chain.run(input_documents=docs, question=query)

' This model has much less image-specific inductive bias than CNNs. In CNNs, locality, two-dimensional neighborhood structure, and translation equivariance are baked into each layer throughout the whole model. In ViT, only MLP layers are local and translationally equivariant, while the self-attention layers are global. The two-dimensional neighborhood structure is used very sparingly in the beginning of the model by cutting the image into patches and at fine-tuning time for adjusting the position embeddings for images of different resolution.'

In [ ]:
query = "What is the conclousion of the paper?"
docs = pdf_embeddings.similarity_search(query)
chain.run(input_documents=docs, question=query)

' The conclusion of the paper is that while Transformers have enabled the training of models of unprecedented size, convolutional architectures remain dominant in computer vision. Multiple works have tried combining CNN-like architectures with self-attention, but the latter models have not yet been effectively scaled on modern hardware accelerators. Therefore, in large-scale image recognition, classic ResNet-like architectures are still state of the art.'

In [ ]:
query = "How old is the first author?"
docs = pdf_embeddings.similarity_search(query)
chain.run(input_documents=docs, question=query)

" I don't know."

## Credit:

ChatGPT for YOUR OWN PDF files with LangChain
https://www.youtube.com/watch?v=TLf90ipMzfE&t=726s